# Demonstrate Inverse Function Sampling
Assuming that for the (continuous) distribution we wish to sample, we have the CDF, $F(x)$, and, we can evaluate its inverse, then
$$
X \sim F^{-1}(\xi), \xi \sim U(0,1)
$$
has the correct distribution.

In [ ]:
using Random
using Plots
using QuadGK
using Roots

In [ ]:
# for readability, these are good settings to use
default(xtickfontsize=14,  ytickfontsize=14, 
    guidefontsize=14, legendfontsize=12, lw=2,ms=8)

# Example 1
Given the density,
$$
f(x) = 3x^2 1_{[0,1]}(x),
$$
the CDF is
$$
F(x) = \begin{cases}
0 & x\leq 0
x^3 & 0<x\leq 1
1 & x>1
\end{cases}
$$
so the relevant inverse we need is
$$
F^{-1}(y)=y^{1/3}
$$

In [ ]:
function inv_sample()
    ξ = rand();
    X = cbrt(ξ);
    return  X
end

In [ ]:
n = 10^4;

Random.seed!(100);
samples = [inv_sample() for _ in 1:n];

In [ ]:
xx = LinRange(0,1,100);
yy = @. 3 * xx^2;

histogram(samples, label = "Samples", norm = :pdf,legend=:topleft)
plot!(xx, yy, label="Density")
xlabel!("x")
ylabel!("Probability")

# Example 2
Suppose we are only given the __unnormalized__ PDF,
$$
f(x) \propto \exp(-(x^2 -1)^2)1_{[-2,2]}(x)
$$
To use inverse function sampling on this problem, we will first need to, numerically, compute the normalization, $Z$,
such that
$$
\int Z^{-1}\exp(-(x^2 -1)^2)1_{[-2,2]}(x)dx = 1.
$$
So
$$
Z = \int_{-2}^2 \exp(-(x^2 -1)^2) dx.
$$

In [ ]:
Z = quadgk(x->exp(-(x^2-1)^2), -2,2)[1];
@show Z;

Note that extract the first return with the `[1]`, as teh second value is an estimate of the error, which we do not need right now.

In [ ]:
# define the density
f = x->exp(-(x^2-1)^2)/Z;

In [ ]:
xx = LinRange(-2,2,100);
plot(xx, f.(xx), label="Density")
xlabel!("x")
ylabel!("Probability")

Observe that the density has two peaks.  This is a __multi-modal__ distribution.  This is also called a double well distribution because $(x^2-1)^2$ is like a potential energy with two _wells_ at $x=\pm 1$.

Next, we construct the numerical CDF,
$$
F(x) = \int_{-2}^x f(t)dt
$$
Observe that, as implemented, this requires us to perform an integral at each value of $x$.

In [ ]:
F = x-> quadgk(f,-2,x)[1];

In [ ]:
xx = LinRange(-2,2,100);
plot(xx, F.(xx), label="CDF", legend=:topleft)
xlabel!("x")
ylabel!("Probability")

To compute $F^{-1}$, we will apply the Bisection method to
$$
F(x) - y = 0
$$
for a given value of $y$.

In [ ]:
function Finv(y)
    x = find_zero(x->F(x) - y, (-2,2), Bisection());
    return x
end

In [ ]:
function double_well_inv_sample()
    ξ = rand();
    X = Finv(ξ);
    return X;
end

In [ ]:
n = 10^4;

Random.seed!(100);
samples = [double_well_inv_sample() for _ in 1:n];

In [ ]:
xx = LinRange(-2,2,100);
histogram(samples, label="Samples", norm =:pdf)
plot!(xx, f.(xx), label="Density")
xlabel!("x")
ylabel!("Probability")